In [ ]:
!pip install konlpy

     |████████████████████████████████| 19.4 MB 1.2 MB/s 
     |████████████████████████████████| 448 kB 52.8 MB/s 
     |████████████████████████████████| 86 kB 4.8 MB/s 
  Attempting uninstall: beautifulsoup4
    Found existing installation: beautifulsoup4 4.6.3
    Uninstalling beautifulsoup4-4.6.3:
      Successfully uninstalled beautifulsoup4-4.6.3


In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import re
import json
import os
import tqdm

from konlpy.tag import Okt

import sklearn
from sklearn.preprocessing import LabelEncoder

from sklearn.metrics import log_loss, accuracy_score,f1_score
import tensorflow as tf
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
import pandas as pd

train = pd.read_csv("/content/drive/MyDrive/dacon 기후기술분류 ai 데이터/train.csv")
labels = pd.read_csv("/content/drive/MyDrive/dacon 기후기술분류 ai 데이터/labels_mapping.csv")

In [ ]:
test = pd.read_csv("/content/drive/MyDrive/dacon 기후기술분류 ai 데이터/test.csv")
sample_submission=pd.read_csv("/content/drive/MyDrive/dacon 기후기술분류 ai 데이터/sample_submission.csv")

In [ ]:
print(train.columns)

Index(['index', '제출년도', '사업명', '사업_부처명', '계속과제여부', '내역사업명', '과제명', '요약문_연구목표',
       '요약문_연구내용', '요약문_기대효과', '요약문_한글키워드', '요약문_영문키워드', 'label'],
      dtype='object')


In [ ]:
#해당 baseline 에서는 과제명 columns만 활용했습니다.
#다채로운 변수 활용법으로 성능을 높여주세요!
#train=train[['과제명','label']]
#test=test[['과제명']]

In [ ]:
#1. re.sub 한글 및 공백을 제외한 문자 제거
#2. okt 객체를 활용해 형태소 단위로 나눔
#3. remove_stopwords로 불용어 제거 
def preprocessing(text, okt, remove_stopwords=False, stop_words=[]):
    text=re.sub("[^가-힣ㄱ-ㅎㅏ-ㅣ]","", text)
    word_text=okt.morphs(text, stem=True)
    if remove_stopwords:
        word_review=[token for token in word_text if not token in stop_words]
    return word_review

stop_words=['은','는','이','가', '하','아','것','들','의','있','되','수','보','주','등','한']
okt=Okt()
clean_train_text=[]
clean_test_text=[]

In [ ]:
print(test.columns)

Index(['index', '제출년도', '사업명', '사업_부처명', '계속과제여부', '내역사업명', '과제명', '요약문_연구목표',
       '요약문_연구내용', '요약문_기대효과', '요약문_한글키워드', '요약문_영문키워드'],
      dtype='object')


In [ ]:
train=train.dropna()

In [ ]:
train=train.reset_index()

In [ ]:
train=train.drop(['level_0'],axis=1)
train

,index,제출년도,사업명,사업_부처명,계속과제여부,내역사업명,과제명,요약문_연구목표,요약문_연구내용,요약문_기대효과,요약문_한글키워드,요약문_영문키워드,label
0,0,2016,농업기초기반연구,농촌진흥청,신규,농산물안전성연구,유전정보를 활용한 새로운 해충 분류군 동정기술 개발,○ 새로운 해충분류군의 동정기술 개발 및 유입확산 추적,(가) 외래 및 돌발해충의 발생조사 및 종 동정\n\n\n ○ 대상해충 : 최...,"○ 새로운 돌발 및 외래해충의 신속, 정확한 동정법 향상\n\n\n○ 돌발 및 외래...","뉴클레오티드 염기서열, 분자마커, 종 동정, 침샘, 전사체","nucleotide sequence, molecular marker, species...",24
1,1,2019,이공학학술연구기반구축(R&D),교육부,신규,지역대학우수과학자지원사업(1년~5년),대장암의 TRAIL 내성 표적 인자 발굴 및 TRAIL 반응 예측 유전자 지도 구축...,최종목표: TRAIL 감수성 표적 유전자를 발굴하고 내성제어 기전을 연구. 발굴된...,1차년도\n1) Microarray를 통한 선천적 TRAIL 내성 표적 후보 유전자...,"1) TRAIL 내성 특이적 표적분자를 발굴하고, 이를 이용한 TRAIL 효과 증진...","대장암,항암제 내성,세포사멸,유전자발굴","TRAIL,Colorectal cancer,TRAIL resistance,Apopt...",0
2,2,2016,중소기업기술혁신개발,중소기업청,신규,혁신기업기술개발,비목질계 셀룰로오스 식물자원을 활용한 기능성 부직포 및 고부가가치 뷰티케어 faci...,* 식물계자원 정련 및 최적 신서란 파이버 기초연구 개발\n\n\n* 소비자 및 바...,* 식물계자원 정련 및 최적 신서란 파이버 기초연구 개발\n\n\n- Tencel/...,* 국내 독자적인 비목질계 셀룰로오스 자원의 파이버 및 부직포 제조 등의 기술 확보...,"기능성 샐룰로오스 파이버,천연섬유,기능성 부직포,뷰티&amp;케어 제품,미용 솜","functional cellulose fiber,natural fiber,funct...",0
3,3,2018,창업성장기술개발(R&D),중소벤처기업부,신규,창업사업화연계과제,소화기 암 진단용 분자영상 형광프로브 개발,# 암특이적 바이오마커 발굴 및 바이오마커에 대한 프로브 개발\n\n\n1) 소화...,# 소화기 암 진단용 분자영상 형광프로브 개발\n\n\n- 국소 도포형 소화기 암 ...,# 암 진단기술의 차별성: 소화기 암 특이 프로브 개발\n\n\n- 최근 체외진단시...,"분자 진단,형광 조영제,프로브,항체,대장암","Molecular diagnosis,Fluorescence,probe,antibod...",0
4,4,2016,이공학개인기초연구지원,교육부,계속,기본연구지원,위암환자의 항암제반응예측을 위한 BRCA 발현검사,"수술이 불가능한 위암환자는 생존기간은 10개월 안팎에 지나지 않고, 항암화학요법에 ...",-In situ hybridization 검사의 정확성을 확인하기 위해 위암세포주 ...,-본 연구는 파라핀보관조직에서 in situ hybridization로 mRNA 및...,"BRCA,제자리부합법,조직미세배열,마이크로RNA,위암,항암제반응,젊은 연령/가족성 위암","BRCA,Insituhybridization,tissuemicroarray,micr...",0
...,...,...,...,...,...,...,...,...,...,...,...,...,...
171133,174299,2016,개인연구지원,미래창조과학부,신규,중견연구,혈관내막 증식을 치료하기 위한 표적분자 발굴,"증식성 혈관질환과 염증반응과의 관련성이 높으나, 혈관평활근세포의 탈분화를 유도하는 ...",● 1차년: 혈관내막 증식과 관련된 표적분자 발굴 및 염증세포의 활성과 분화에 대한...,심혈관 질환 발생의 새로운 기전 규명뿐만 아니라 제어할 수 있는 새로운 방법을 탐...,"혈관내막 증식,혈관 개조,혈관평활근세포,염증세포,혈관염증,혈관평활근세포 탈분화,표적분자","Intimahyperplasia,Vascularremodeling,Vasculars...",0
171134,174300,2016,개인연구지원,미래창조과학부,신규,중견연구,삼킴곤란 개선을 위한 물성조절 식품시스템에서의 유동층과립 복합소재의 유변물성 변화 ...,삼킴곤란을 겪고 있는 고령자의 안전한 식품섭취는 삼킴에 적합한 유변물성을 가진 물성...,본 연구에서는 여러 가지 유변물성 측정기술을 이용하여 고령자의 삼킴곤란 개선을 위해...,○ 그 동안 연구되지 않은 다양한 물성조절 검복합 소재의 과립제형 및 유변물성 연구...,"삼킴곤란,유변물성,유동층과립,물리적 특성,검복합물","Swallowingdifficulty,Rheologicalproperty,Fludi...",0
171135,174301,2018,이공학학술연구기반구축(R&D),교육부,계속,이공분야학문후속세대양성_대통령POST_DOC펠로우쉽,식물 유전체 정보 고도화 및 핵심 유전자 군 진화연구,"지난 10년여 동안 100여종에 대한 식물 유전체 정보가 완성되어 공개되었지만, 애...",○ 공개된 식물 유전체 정보 및 대상 유전자 군 선발\n\n - 공개된 식물 유...,- 본 과제에서 개발한 annotation tool은 유전체 고도화 부분에서 가장 ...,"유전자 정보 고도화,대상 유전자 군 재분석,비교 유전체 진화 분석,공개 식물 유전체...","Gene model improvement,Target gene family re-a...",0
171136,174302,2019,수요자맞춤형육종자원대량신속발굴기술개발(R&D),농촌진흥청,신규,유용형질 대량 특성평가를 통한 육종모본 발굴,"콩 유전자원 내건성, 내습성 대량평가 체계구축 및 자원선발",1) 내건 및 내습성 유전자원 수집\n\n2) 내건성 자원평가 체계구축\n\n3) ...,[1차 년도(2019)] \n\n1) 내건 및 내습성 유전자원 수집\n\n○ 기존 ...,○ 내건성 및 내습성 특성평가 체계구축 및 우수자원 발굴로 내재해성 콩 품종육성 자...,"콩, 가공적성, 병 저항성, 내재해성, 콩모자이크바이러스","soybean, food application, resistance, biotic-...",23


In [ ]:
train['합계']=str(0)
train['합계']=train['사업명']+" "+train['사업_부처명']+" "+train["과제명"]+" "+train["요약문_한글키워드"]
train['합계']

0         농업기초기반연구 농촌진흥청 유전정보를 활용한 새로운 해충 분류군 동정기술 개발 뉴클...
1         이공학학술연구기반구축(R&D) 교육부 대장암의 TRAIL 내성 표적 인자 발굴 및 ...
2         중소기업기술혁신개발 중소기업청 비목질계 셀룰로오스 식물자원을 활용한 기능성 부직포 ...
3         창업성장기술개발(R&D) 중소벤처기업부 소화기 암 진단용 분자영상 형광프로브 개발 ...
4         이공학개인기초연구지원 교육부 위암환자의 항암제반응예측을 위한 BRCA 발현검사 BR...
                                ...                        
171133    개인연구지원 미래창조과학부 혈관내막 증식을 치료하기 위한 표적분자 발굴 혈관내막 증...
171134    개인연구지원 미래창조과학부 삼킴곤란 개선을 위한 물성조절 식품시스템에서의 유동층과립...
171135    이공학학술연구기반구축(R&D) 교육부 식물 유전체 정보 고도화 및 핵심 유전자 군 ...
171136    수요자맞춤형육종자원대량신속발굴기술개발(R&D) 농촌진흥청 콩 유전자원 내건성, 내습...
171137    이공학학술연구기반구축(R&D) 교육부 광학적 위상 제어 기법을 적용한 디지털 홀로그...
Name: 합계, Length: 171138, dtype: object

In [ ]:
test['합계']=str(0)
test['합계']=test['사업명']+" "+test['사업_부처명']+" "+test["과제명"]+" "+test["요약문_한글키워드"]
test['합계']

0        경제협력권산업육성 산업통상자원부 R-FSSW 기술 적용 경량 차체 부품 개발 및 품...
1        개인기초연구(과기정통부)(R&D) 과학기술정보통신부 다입자계를 묘사하는 편미분방정식...
2        개인기초연구(미래부) 과학기술정보통신부 저출생체중아 가족을 위한 지역사회중심의 퇴원...
3        산학연협력기술개발(R&D) 중소벤처기업부 폴리싱용 5축 CNC 정밀 마이크로 시스템...
4        이공학학술연구기반구축(R&D) 교육부 다결정재료의 미세조직구조를 고려한 확률론적 응...
                               ...                        
43571    고부가가치식품기술개발 농림축산식품부 포장용 인디케이팅 잉크 제조기술 개발 및 인쇄 ...
43572    산업기술국제협력 산업통상자원부 루시아-신속/정확/저렴한 현장진단용 혁신적 혈액진단 ...
43573    국제핵융합실험로공동개발사업(일반+기금) 미래창조과학부 ITER 삼중수소 저장·공급 ...
43574    집단연구지원(R&D) 과학기술정보통신부 무결정립계 금속 초박막의 제조 및 이를 활용...
43575    산업집적지경쟁력강화 산업통상자원부 산업용 M30 SUS Bolt의 생산성 향상 제조...
Name: 합계, Length: 43576, dtype: object

In [ ]:
#시간이 많이 걸립니다.
for text in tqdm.tqdm(train['합계']):
  try:
    clean_train_text.append(preprocessing(text, okt, remove_stopwords=True, stop_words=stop_words))
  except:
    clean_train_text.append([])

100%|██████████| 171138/171138 [3:35:11<00:00, 13.26it/s]


In [ ]:
for text in tqdm.tqdm(test['합계']):
    try:
      clean_test_text.append(preprocessing(text, okt, remove_stopwords=True, stop_words=stop_words))
    except:
      clean_test_text.append([])

100%|██████████| 43576/43576 [1:09:26<00:00, 10.46it/s]


In [ ]:
#텐서플로의 전처리 모듈을 활용해 토크나이징 객체를 만든 후 인덱스 벡터로 전환
tokenizer=Tokenizer()
tokenizer.fit_on_texts(clean_train_text)

train_sequences=tokenizer.texts_to_sequences(clean_train_text)
test_sequences=tokenizer.texts_to_sequences(clean_test_text)
word_vocab=tokenizer.word_index

#패딩 처리
train_inputs=pad_sequences(train_sequences, maxlen=40, padding='post')
test_inputs=pad_sequences(test_sequences, maxlen=40, padding='post')

print(train_inputs.shape)
print(test_inputs.shape)

(171138, 40)
(43576, 40)


In [ ]:

data_configs={}
data_configs['vocab']=word_vocab
data_configs['vocab_size'] = len(word_vocab)+1

In [ ]:
#파라미터 설정
vocab_size =data_configs['vocab_size']
embedding_dim = 32
max_length = 40
oov_tok = "<OOV>"

#가벼운 NLP모델 생성
model = tf.keras.Sequential([
    tf.keras.layers.Embedding(vocab_size, embedding_dim, input_length=max_length),
    tf.keras.layers.GlobalAveragePooling1D(),
    tf.keras.layers.Dense(128, activation='relu'),
    tf.keras.layers.Dense(46, activation='softmax')
])

# compile model
model.compile(loss='sparse_categorical_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])

# model summary
print(model.summary())

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding (Embedding)        (None, 40, 32)            1468576   
_________________________________________________________________
global_average_pooling1d (Gl (None, 32)                0         
_________________________________________________________________
dense (Dense)                (None, 128)               4224      
_________________________________________________________________
dense_1 (Dense)              (None, 46)                5934      
Total params: 1,478,734
Trainable params: 1,478,734
Non-trainable params: 0
_________________________________________________________________
None


In [ ]:
labels=np.array(train['label'])
len(set(labels))

46

In [ ]:
from tensorflow.keras.callbacks import EarlyStopping
es = EarlyStopping(monitor='val_loss', verbose=1, patience=2, mode='min')

In [ ]:
# fit model
num_epochs = 30
history = model.fit(train_inputs, labels, 
                    epochs=num_epochs, verbose=2, 
                    validation_split=0.2,callbacks = [es])

Epoch 1/30
4279/4279 - 35s - loss: 0.7476 - accuracy: 0.8381 - val_loss: 0.5293 - val_accuracy: 0.8592
Epoch 2/30
4279/4279 - 34s - loss: 0.4042 - accuracy: 0.8869 - val_loss: 0.4144 - val_accuracy: 0.8861
Epoch 3/30
4279/4279 - 34s - loss: 0.2907 - accuracy: 0.9135 - val_loss: 0.3736 - val_accuracy: 0.8951
Epoch 4/30
4279/4279 - 34s - loss: 0.2272 - accuracy: 0.9303 - val_loss: 0.3684 - val_accuracy: 0.9007
Epoch 5/30
4279/4279 - 34s - loss: 0.1853 - accuracy: 0.9425 - val_loss: 0.3701 - val_accuracy: 0.9058
Epoch 6/30
4279/4279 - 34s - loss: 0.1550 - accuracy: 0.9511 - val_loss: 0.3921 - val_accuracy: 0.9066
Epoch 00006: early stopping


In [ ]:
#평가지표가 Macro F1이기에 확률값으로 결과를 내면 안됩니다.
pred=model.predict(test_inputs)
pred=tf.argmax(pred, axis=1)

sample_submission['label']=pred

sample_submission

,index,label
0,174304,0
1,174305,0
2,174306,0
3,174307,0
4,174308,0
...,...,...
43571,217875,0
43572,217876,0
43573,217877,2
43574,217878,0


In [ ]:
sample_submission.to_csv('/content/drive/MyDrive/dacon 기후기술분류 ai 데이터/baseline.csv', index=False) #0.64

In [ ]:
from tensorflow.keras.optimizers import Adam
import tensorflow as tf
from tensorflow.keras.layers import Embedding, LSTM, Dense, Bidirectional, Dropout,Conv1D, GlobalMaxPooling1D, MaxPooling1D, SpatialDropout1D

#CNN_BiLSTM_preprocessing0 -> Embedding dim 좀 작게(32) 정도로 해도 될듯?
model = tf.keras.Sequential([
    Embedding(vocab_size, 100, input_length =max_length),
    Bidirectional(LSTM(128, return_sequences=True)),
    Conv1D(filters=128, kernel_size=5, padding='same', activation='relu'),
    GlobalMaxPooling1D(),
    Dropout(0.25),
    # Bidirectional(LSTM(100), return_sequences=True),
    Dense(128, activation='relu'),
    Dense(46, activation='softmax')
])

model.compile(loss='sparse_categorical_crossentropy', optimizer=Adam(learning_rate=0.001), metrics=['accuracy'])

In [ ]:
history = model.fit(train_inputs, labels, validation_split=0.2, epochs=20, batch_size=128 ,callbacks = [es])

Epoch 1/20
1070/1070 [==============================] - 357s 331ms/step - loss: 0.6840 - accuracy: 0.8470 - val_loss: 0.4551 - val_accuracy: 0.8786
Epoch 2/20
1070/1070 [==============================] - 352s 329ms/step - loss: 0.3674 - accuracy: 0.8948 - val_loss: 0.3859 - val_accuracy: 0.8934
Epoch 3/20
1070/1070 [==============================] - 353s 330ms/step - loss: 0.2642 - accuracy: 0.9190 - val_loss: 0.3558 - val_accuracy: 0.9036
Epoch 4/20
1070/1070 [==============================] - 353s 330ms/step - loss: 0.1965 - accuracy: 0.9381 - val_loss: 0.3686 - val_accuracy: 0.9100
Epoch 5/20
1070/1070 [==============================] - 353s 330ms/step - loss: 0.1560 - accuracy: 0.9500 - val_loss: 0.3946 - val_accuracy: 0.9117
Epoch 00005: early stopping


In [ ]:
model.summary()

Model: "sequential_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_2 (Embedding)      (None, 40, 100)           4589300   
_________________________________________________________________
bidirectional_1 (Bidirection (None, 40, 256)           234496    
_________________________________________________________________
conv1d_1 (Conv1D)            (None, 40, 128)           163968    
_________________________________________________________________
global_max_pooling1d_1 (Glob (None, 128)               0         
_________________________________________________________________
dropout_1 (Dropout)          (None, 128)               0         
_________________________________________________________________
dense_4 (Dense)              (None, 128)               16512     
_________________________________________________________________
dense_5 (Dense)              (None, 46)               

In [ ]:
#평가지표가 Macro F1이기에 확률값으로 결과를 내면 안됩니다.
pred=model.predict(test_inputs)
pred=tf.argmax(pred, axis=1)

sample_submission['label']=pred

sample_submission

,index,label
0,174304,0
1,174305,0
2,174306,0
3,174307,0
4,174308,1
...,...,...
43571,217875,0
43572,217876,0
43573,217877,2
43574,217878,0


In [ ]:
sample_submission.to_csv('/content/drive/MyDrive/dacon 기후기술분류 ai 데이터/baseline1.csv', index=False)